已连接到 base (Python 3.12.3)

In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler

import pandas as pd
import pickle
from tqdm import tqdm

# # 下载数据集并保存至本地
# from ucimlrepo import fetch_ucirepo 
# rt_iot2022 = fetch_ucirepo(id=942) 
# pickle.dump(rt_iot2022, open("rt_iot2022.pkl", "wb"))

# 从本地加载数据集
rt_iot2022 = pickle.load(open("rt_iot2022.pkl", "rb"))

# data (as pandas dataframes)
X = rt_iot2022.data.features
y = rt_iot2022.data.targets

# 删除 service 属性
X = X.drop(columns=["service"])
print("初始 X 形状：", X.shape)

# 将 proto 属性独热编码
X = pd.get_dummies(X, columns=["proto"])
print("对 proto 独热编码后的形状：", X.shape)
print("对 proto 独热编码后的 head():", X.head())

# 使用十折交叉验证划分训练集和测试集
kf = KFold(n_splits=10, shuffle=True)

初始 X 形状： (123117, 82)
对 proto 独热编码后的形状： (123117, 84)
对 proto 独热编码后的 head():    id.orig_p  id.resp_p  flow_duration  fwd_pkts_tot  bwd_pkts_tot  \
0      38667       1883      32.011598             9             5   
1      51143       1883      31.883584             9             5   
2      44761       1883      32.124053             9             5   
3      60893       1883      31.961063             9             5   
4      51087       1883      31.902362             9             5   

   fwd_data_pkts_tot  bwd_data_pkts_tot  fwd_pkts_per_sec  bwd_pkts_per_sec  \
0                  3                  3          0.281148          0.156193   
1                  3                  3          0.282277          0.156821   
2                  3                  3          0.280164          0.155647   
3                  3                  3          0.281593          0.156440   
4                  3                  3          0.282111          0.156728   

   flow_pkts_per_sec  ...   

In [2]:
all_accuracies = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    knn = KNeighborsClassifier(n_neighbors=3)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    all_accuracies.append(accuracy)

print("KNN 准确率：", sum(all_accuracies) / len(all_accuracies))

/usr/local/miniforge3/lib/python3.12/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/miniforge3/lib/python3.12/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/miniforge3/lib/python3.12/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/miniforge3/lib/python3.12/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please

KNN 准确率： 0.9907811212749245


In [5]:
all_accuracies = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    dt = DecisionTreeClassifier()
    dt.fit(X_train, y_train)
    y_pred = dt.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    all_accuracies.append(accuracy)

print("CART 决策树准确率：", sum(all_accuracies) / len(all_accuracies))

CART 决策树准确率： 0.997709488980863


In [6]:
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    rf = RandomForestClassifier()
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    all_accuracies.append(accuracy)

print("随机森林准确率：", sum(all_accuracies) / len(all_accuracies))

/usr/local/miniforge3/lib/python3.12/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/usr/local/miniforge3/lib/python3.12/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/usr/local/miniforge3/lib/python3.12/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/usr/local/miniforge3/lib/python3.12/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the

随机森林准确率： 0.9981115502789442


In [12]:
class DNN(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(DNN, self).__init__()
        self.hidden_layers = nn.ModuleList()
        for i in range(len(hidden_sizes) - 1):
            self.hidden_layers.append(nn.Linear(hidden_sizes[i], hidden_sizes[i + 1]))
        self.output_layer = nn.Linear(hidden_sizes[-1], output_size)

    def forward(self, x):
        for layer in self.hidden_layers:
            x = torch.relu(layer(x))
        x = self.output_layer(x)
        return x


hidden_sizes = [X.shape[1], 64]


all_accuracies = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model = DNN(X.shape[1], hidden_sizes, y["Attack_type"].nunique())
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    # 标准化数据数据
    scaler = StandardScaler()
    X_train_std = scaler.fit_transform(X_train)
    X_test_std = scaler.transform(X_test)
    # 使用类别索引进行特征类别转化
    y_train_cat = y_train["Attack_type"].astype("category").cat.codes
    y_test_cat = y_test["Attack_type"].astype("category").cat.codes
    # # print(y_train_cat.head())
    # print(y_test_cat.head())
    # # 输出类别总数
    # print(y_train_cat.nunique())

    # 转为 PyTorch 的 Tensor 格式
    X_train_tensor = torch.tensor(X_train_std, dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test_std, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train_cat.values, dtype=torch.long)
    y_test_tensor = torch.tensor(y_test_cat.values, dtype=torch.long)

    # 创建数据加载器
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

    # 训练模型
    num_epochs = 10
    for epoch in tqdm(range(num_epochs)):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # 在测试集上评估模型
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
        _, predicted = torch.max(outputs, 1)
        accuracy = (predicted == y_test_tensor).sum().item() / len(y_test)
        all_accuracies.append(accuracy)
    print(f"准确率：{accuracy}")

print("DNN 准确率（总体）：", sum(all_accuracies) / len(all_accuracies))

100%|██████████| 10/10 [00:19<00:00,  1.96s/it]


准确率：0.994314489928525


100%|██████████| 10/10 [00:19<00:00,  1.93s/it]


准确率：0.9939083820662769


100%|██████████| 10/10 [00:20<00:00,  2.03s/it]


...

准确率：0.9943140281049468


100%|██████████| 10/10 [00:20<00:00,  2.05s/it]


准确率：0.9913085858175615


100%|██████████| 10/10 [00:20<00:00,  2.02s/it]

准确率：0.9928519210462188
DNN 准确率（总体）： 0.993705153301941
